In [1]:
from initialization.wp_get_page import WikipediaPage
from initialization.wp_structured_text import WikipediaStructuredText
from initialization.wp_structured_diffs import Diff
import difflib
from setup.db_trie import db
from lakat.submit import content_submit
from config.scrape_cfg import WIKIPEDIA_API_URL, EXAMPLE_ARTICLE_TITLE
from config.bucket_cfg import DEFAULT_ATOMIC_BUCKET_SCHEMA, DEFAULT_MOLECULAR_BUCKET_SCHEMA, DEFAULT_NAME_RESOLUTION_BUCKET_SCHEMA, BUCKET_ID_TYPE_NO_REF, BUCKET_ID_TYPE_WITH_ID_REF
from utils.signing.sign import get_public_key_from_file
from utils.serialize import unserialize, serialize
from initialization.test_branch_creation import branch_creation
from initialization.test_wp_submissions import first_submit, submit_new_edit



In [2]:
# db.close()

In [3]:

# db.close()
article_title = EXAMPLE_ARTICLE_TITLE
print('article_title: ', article_title)

wp = WikipediaPage(WIKIPEDIA_API_URL)
edit_history = wp.load_content_from_batches(
    article_title, 0, 105, download_if_not_exist=True)
edit_history.reverse()

key_file_prefix="lakat"
public_key = get_public_key_from_file(key_file_prefix=key_file_prefix)
# create a name registry NR (optional)

article_title:  Compartmental models in epidemiology


In [4]:
branchId = branch_creation(public_key=public_key, verbose=False)

In [4]:
new_part_id_to_bucket_id, res = first_submit(edit=edit_history[0], public_key=public_key, branchId=branchId)


In [6]:
article_id = res["bucket_ids"][-1] ## hack TODO! Change this.

In [7]:
new_part_id_to_bucket_id, res = submit_new_edit(
    public_key, 
    branchId, prev_edit=edit_history[0], new_edit=edit_history[1], article_id=article_id, part_id_to_bucket_id=new_part_id_to_bucket_id)

In [8]:

new_part_id_to_bucket_id, res = submit_new_edit(
    public_key, 
    branchId, prev_edit=edit_history[1], new_edit=edit_history[2], article_id=article_id, part_id_to_bucket_id=new_part_id_to_bucket_id)